In [1]:
# define class and a track for I/O
import numpy as np
import pandas as pd

In [2]:
# take 2022 data for example
df = pd.read_excel(r'C:\Users\22785\Downloads\y2022n.xlsx')
df.head()

,Unnamed: 0.1,Unnamed: 0,id,channel,context,conversationId,entity,flowCategory,input,intent,...,intent_2_intent,intent_2_confidence,lang,old_id_format,senderId,output,if_button,conv_gp,convnum_bf,convnum_aft
0,0,18,456986567150842270020220301033617844,myflight,"{'24hour': None, 'airline': [], 'detected_airl...",451369e4-8d05-442a-abb4-6880a14e9d7f,"[{'entity': 'brand_name', 'location': [0, 8], ...",shop_and_restaurant,Mcdonald,"[{'intent': 'SNR-RESTAURANT', 'confidence': 0....",...,SNR-SHOP,0.678368,EN,20220301033619275451369e4-8d05-442a-abb4-6880a...,4569865671508422700,[],0,7,2,1
1,1,20,452823844815765100020220301034109256,myflight,"{'24hour': None, 'airline': [], 'detected_airl...",6e32a3ef-4d15-4622-b627-4a36da4ac27d,"[{'entity': 'airline', 'location': [4, 7], 'va...",shop_and_restaurant_btn,*Restaurants,"[{'intent': 'SNR-RESTAURANT', 'confidence': 1}...",...,AIRLINE-GET_AIRLINE_INFO,0.000000,EN,202203010341108676e32a3ef-4d15-4622-b627-4a36d...,4528238448157651000,['Question: Please select one of the following...,1,8,21,20
2,2,21,452823844815765100020220301034117224,myflight,"{'24hour': None, 'airline': [], 'detected_airl...",6e32a3ef-4d15-4622-b627-4a36da4ac27d,"[{'entity': 'airline', 'location': [4, 7], 'va...",shop_and_restaurant_btn,*Western Food & Drink,"[{'intent': 'SNR-RESTAURANT', 'confidence': 0....",...,AIRLINE-GET_AIRLINE_INFO,0.271309,EN,202203010341184486e32a3ef-4d15-4622-b627-4a36d...,4528238448157651000,['This is the list of Western Restaurants shop...,1,8,21,20
3,3,22,452823844815765100020220301034154530,myflight,"{'24hour': None, 'airline': [], 'detected_airl...",6e32a3ef-4d15-4622-b627-4a36da4ac27d,NaN,shop_and_restaurant_btn,*More Choices,NaN,...,NaN,NaN,EN,202203010341546666e32a3ef-4d15-4622-b627-4a36d...,4528238448157651000,['This is the list of Western Restaurants shop...,1,8,21,20
4,4,23,452823844815765100020220301034228956,myflight,"{'24hour': None, 'airline': [], 'detected_airl...",6e32a3ef-4d15-4622-b627-4a36da4ac27d,"[{'entity': 'airline', 'location': [4, 7], 'va...",others_btn,*Other Queries,"[{'intent': 'AIRLINE-GET_AIRLINE_INFO', 'confi...",...,OTH-DIRTY_WORD,0.297948,EN,202203010342303136e32a3ef-4d15-4622-b627-4a36d...,4528238448157651000,"['Question: What else can I help you to find?,...",1,8,21,20


In [3]:
# get all convid as a set
convid_set = set(df['conversationId'])
convid_set = list(convid_set)

In [4]:
# define conversation class
class conversation:
    def __init__(self, dataframe, convid):
        self.convid = convid
        self.content = dataframe[dataframe['conversationId']==convid ]# content must be a dataframe
    
    def get_channel(self):
        return set(self.content['channel'])

    def get_interact_num(self):
        return self.content.shape[0]
    
    def get_language(self):
        return set(self.content['lang'])
    
    def intent_list(self):
        return list(self.content['intent_1_intent'])
    
    def intent1_list(self):
        return list(self.content['intent_1_intent'])
    

In [5]:
# define conversation interaction class
# actually linked list
class conversation_node:
    def __init__(self,value):
        self.value = value
        self.next = None
    
    def get_convid(self):
        return self.value['conversationId'][0]
    
    def get_intent(self):
        return self.value['intent_1_intent'][0]
    
    def get_input(self):
        return self.value['input'][0]
    
    def get_output(self):
        return self.value['output'][0]
    
    def save_info_in_dict(self):
        dic = {}
        dic['conversationId'] = self.get_convid()
        dic['content'] = self.get_intent()
        dic['input'] = self.get_input()
        dic['output'] = self.get_output()
        dic = [dic]
        return pd.DataFrame(dic)
        
class conversation_linked_list: 
    def __init__(self,value):
        new_node = conversation_node(value)
        self.head = new_node
        self.tail = new_node
        self.length = 1

    def print_list(self): # print that linked list
        temp = self.head
        while temp is not None:
            print('{: <9} {: <9} {: <9}'.format(' ',' ','content'))
            print('         --------------------')
            print('intent={: <9}|input={: <6}|output={: <9}'.format(str(temp.get_intent()),str(temp.get_input()),str(temp.get_output())))
            print('-----------------------------')
            temp = temp.next

    def print_content(self):
        temp = self.head
        ct = self.head.save_info_in_dict()
        temp = temp.next
        while temp is not None:
            ct = pd.concat([ct,temp.save_info_in_dict()])
            temp = temp.next
        return ct

    # O(1)
    def append(self,value): # create new node and add it to the end
        new_node = conversation_node(value)
        if self.head is None: # if empty linked list, let the new node initialize
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node
        self.length += 1 
        return True # this funcation will be call again, so we need a return
    
    def popFirst(self): # pop the first item in the linkedlist
        if self.length == 0:
            return None
        temp = self.head # create this because we need a return of poped
        self.head = self.head.next
        temp.next = None
        self.length -= 1
        if self.length == 0:
            self.tail = None
        return temp

In [6]:
# get some info from that data structure(just for use, not find more useful usage)
huge_2022_list = []
for i in range(len(convid_set)):
    conv = conversation(dataframe=df,convid=convid_set[i])
    conv_linked_list = conversation_linked_list(0)

    for i in range(conv.content.shape[0]):
        conv_linked_list.append(value = conv.content.iloc[i:i+1,:].reset_index())

    conv_linked_list.popFirst()
    huge_2022_list.append(conv_linked_list.print_content())

In [7]:
# can get any conversation just typing the index
huge_2022_list[9]

,conversationId,content,input,output
0,c1f0346f-7f4d-44a7-95db-e0b21d657ba0,AIRLINE-GET_AIRLINE_INFO,*Airlines,['Question: Please select the Airline from bel...
0,c1f0346f-7f4d-44a7-95db-e0b21d657ba0,AIRLINE-GET_AIRLINE_INFO,*Other Queries,"['Question: What else can I help you to find?,..."
0,c1f0346f-7f4d-44a7-95db-e0b21d657ba0,AIRLINE-GET_AIRLINE_INFO,May I ask which airline is the Hong Kong port ...,['The following airline information is for you...
0,c1f0346f-7f4d-44a7-95db-e0b21d657ba0,LNF-LOST_AND_FOUND,If I.lost my bag while arrived LONDON,['Please seek for help from Customer Service C...
